## Day 1 — Setup & Imports

# Linear Regression from Scratch
Goal: implement linear regression from scratch with NumPy.
Steps: derive gradients, implement training loop, plot loss.

# Linear Regression (from scratch)
Following Andrew Ng ML Course - Week 1

## Day 2 — Generate & Visualise a Synthetic Linear Dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

# Generate a synthetic linear dataset
X = 2 * np.random.rand(100, 1)     # 100 points between 0 and 2
y = 4 + 3 * X + np.random.randn(100, 1)  # y = 4 + 3x + Gaussian noise

# Plot the data
plt.scatter(X, y, alpha=0.7)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Synthetic Linear Dataset")
plt.show()



## Day 3 — Cost Function & Gradient Descent

**Goal today**
- Define hypothesis and **cost function** \(J(\theta)\)
- Implement **gradient descent**
- Train on your synthetic data and **watch the cost decrease**


In [ ]:
# Prepare design matrix with bias term (1s column)
try:
    X  # reuse from Day 2 if present
    y
except NameError:
    # Fallback: generate the same synthetic dataset if notebook was restarted
    X = 2 * np.random.rand(100, 1)
    y = 4 + 3 * X + np.random.randn(100, 1)

import numpy as np

m = len(y)
X_b = np.c_[np.ones((m, 1)), X]  # shape (m, 2): [1, x]
print("X_b shape:", X_b.shape)

In [ ]:
def compute_cost(X, y, theta):
    """Mean Squared Error cost for linear regression.
    X: (m, n) design matrix (already includes bias column)
    y: (m, 1) targets
    theta: (n, 1) parameters
    """
    m = len(y)
    predictions = X.dot(theta)
    errors = predictions - y
    return (1 / (2 * m)) * np.sum(errors ** 2)

# Quick sanity check
theta_zeros = np.zeros((X_b.shape[1], 1))
print("Initial cost at theta=zeros:", compute_cost(X_b, y, theta_zeros))

In [ ]:
def gradient_descent(X, y, theta, alpha=0.1, num_iters=1000):
    m = len(y)
    J_hist = []
    for _ in range(num_iters):
        predictions = X.dot(theta)
        errors = predictions - y
        gradients = (1 / m) * X.T.dot(errors)
        theta = theta - alpha * gradients
        J_hist.append(compute_cost(X, y, theta))
    return theta, J_hist

# Train
theta_init = np.zeros((X_b.shape[1], 1))
theta_final, J_hist = gradient_descent(X_b, y, theta_init, alpha=0.1, num_iters=1000)
print("Learned theta:", theta_final.ravel())

In [ ]:
# Plot cost curve
import matplotlib.pyplot as plt

plt.plot(J_hist)
plt.xlabel("Iteration")
plt.ylabel("Cost J(θ)")
plt.title("Gradient Descent Convergence")
plt.show()

In [ ]:
# Visualize fitted line vs data
import numpy as np
import matplotlib.pyplot as plt

x_line = np.array([[0.0], [2.0]])
X_line_b = np.c_[np.ones((2, 1)), x_line]
y_line = X_line_b.dot(theta_final)

plt.scatter(X, y, alpha=0.7, label="data")
plt.plot(x_line, y_line, label="fitted line", linewidth=2)
plt.xlabel("x"); plt.ylabel("y"); plt.title("Linear Regression Fit")
plt.legend()
plt.show()